In [3]:
import dataset_utils
import preprocessing_utils
from tqdm import tqdm # loading bar

In [4]:
import json
import pandas as pd

### Load Datasets

In [5]:
def load_dataset(split={"train", "test", "dev"}, domain={"rest", "laptop"}):
    """
    loading dataset as given in .json file
    file needs to be in specified directory
    """
    with open(f'asc/{domain}/{split}.json', 'r') as file:
        dataset = json.load(file)
    return list(dataset.values())

In [6]:
import random

def print_random_element(dataset):
    """
    output one random element from the dataset
    """
    random_element = random.choice(dataset)
    print(random_element)

#### load all subsets of the dataset

In [7]:
rest_train = load_dataset(split="train", domain="rest") + load_dataset(split="dev", domain = "rest")
rest_test = load_dataset(split="test", domain="rest")
laptop_train = load_dataset(split="train", domain="laptop") + load_dataset(split="dev", domain = "laptop")
laptop_test = load_dataset(split="test", domain="laptop")

In [8]:
print_random_element(rest_train)

{'polarity': 'positive', 'term': 'decor', 'id': '3535_2', 'sentence': "The decor is dark, cool and soothing, while the food's presentation is spectacular, considering the low prices."}


### Preprocessing dataset

In [9]:
import re
import string

def clean_text(text):
    """
    Function to clean text data.
    :param text: The input string containing the text to be cleaned.
    :return: Cleaned text.
    """

    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)

    return text

In [10]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Simon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Simon\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Simon\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [11]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """
    Map POS tag to the first character lemmatize() accepts.
    """
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def tokenize_and_lemmatize(text):
    """
    Function to tokenize and lemmatize the input text.
    :param text: The input string containing the text to be processed.
    :return: A list of lemmatized tokens.
    """
    lemmatizer = WordNetLemmatizer()
    tokenized_text = word_tokenize(text)
    lemmatized_text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokenized_text]
    return lemmatized_text


In [12]:
def preprocess_text(text):
    """
    Function to preprocess any input text by first cleaning text (e.g. of numbers,...)
    and then lemmatize and tokenizing the text

    :param text: Input string of text to be processed
    :return: preprocessed text
    """
    cleaned_text = clean_text(text)
    preprocessed_text = tokenize_and_lemmatize(cleaned_text)
    
    return preprocessed_text

In [13]:
def preprocess_dataset(dataset):
    """
    Function to preprocess any subset of the dataset

    :param text: Dataset to be preprocessed
    """
    for element in tqdm(dataset, desc="Processing Text"):
        element['sentence'] = preprocess_text(element['sentence'])

#### preprocessing all subsets

In [11]:
preprocess_dataset(rest_train)
preprocess_dataset(laptop_train)
preprocess_dataset(rest_test)
preprocess_dataset(laptop_test)

Processing Text: 100%|██████████| 638/638 [00:04<00:00, 151.13it/s]


### TFIDF with Logistic Regression

In [12]:
print(rest_train)
print(rest_test)

[{'polarity': 'positive', 'term': 'server', 'id': '1592_0', 'sentence': ['our', 'server', 'be', 'very', 'helpful', 'and', 'friendly']}, {'polarity': 'positive', 'term': 'staff', 'id': '1940_1', 'sentence': ['super', 'friendly', 'and', 'knowledgable', 'staff', 'fabulous', 'bistro', 'fare', 'and', 'a', 'wonderful', 'jazz', 'brunch', 'with', 'great', 'live', 'jazz', 'the', 'chilaquiles', 'be', 'awesome']}, {'polarity': 'positive', 'term': 'jazz brunch', 'id': '1940_0', 'sentence': ['super', 'friendly', 'and', 'knowledgable', 'staff', 'fabulous', 'bistro', 'fare', 'and', 'a', 'wonderful', 'jazz', 'brunch', 'with', 'great', 'live', 'jazz', 'the', 'chilaquiles', 'be', 'awesome']}, {'polarity': 'positive', 'term': 'live jazz', 'id': '1940_3', 'sentence': ['super', 'friendly', 'and', 'knowledgable', 'staff', 'fabulous', 'bistro', 'fare', 'and', 'a', 'wonderful', 'jazz', 'brunch', 'with', 'great', 'live', 'jazz', 'the', 'chilaquiles', 'be', 'awesome']}, {'polarity': 'positive', 'term': 'bistro 

In [26]:
rest_train_sentences = [" ".join(item['sentence']) for item in rest_train]
rest_train_polarities = [item['polarity'] for item in rest_train]

rest_test_sentences = [" ".join(item['sentence']) for item in rest_test]
rest_test_polarities = [item['polarity'] for item in rest_test]

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'sentences' is a list of text data (your reviews)
tfidf_vectorizer = TfidfVectorizer()

all_sentences = rest_test_sentences + rest_test_sentences
tfidf_vectorizer.fit(all_sentences)
X_train = tfidf_vectorizer.transform(rest_train_sentences)
X_test = tfidf_vectorizer.transform(rest_test_sentences)

In [35]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(rest_train_polarities)
y_test = label_encoder.transform(rest_test_polarities)

In [36]:
print(label_encoder.classes_)

['negative' 'neutral' 'positive']


In [37]:
from sklearn.linear_model import LogisticRegression

# Example: Training a Logistic Regression model
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train, y_train)

LogisticRegression()

In [38]:
# Predict on test data
y_pred = log_reg_model.predict(X_test)

1120
1120


In [39]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

    negative       0.61      0.40      0.48       196
     neutral       0.58      0.11      0.19       196
    positive       0.73      0.96      0.83       728

    accuracy                           0.71      1120
   macro avg       0.64      0.49      0.50      1120
weighted avg       0.68      0.71      0.66      1120



### TFIDF with SVM

In [40]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
# Create the SVM model with a linear kernel
svm_model = SVC(kernel='linear', class_weight='balanced')  # 'balanced' automatically adjusts weights inversely proportional to class frequencies

# Train the model
svm_model.fit(X_train, y_train)

SVC(class_weight='balanced', kernel='linear')

In [41]:
# Make predictions
y_pred = svm_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

    negative       0.46      0.64      0.53       196
     neutral       0.37      0.37      0.37       196
    positive       0.84      0.76      0.80       728

    accuracy                           0.67      1120
   macro avg       0.56      0.59      0.57      1120
weighted avg       0.69      0.67      0.68      1120



### Oversampling for Imbalanced Dataset

In [46]:
from imblearn.over_sampling import SMOTE
from collections import Counter

print(f"Original dataset shape: {Counter(y_train)}")

smote = SMOTE()
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print(f"Resampled dataset shape: {Counter(y_train_res)}")

Original dataset shape: Counter({2: 2164, 0: 805, 1: 633})
Resampled dataset shape: Counter({2: 2164, 0: 2164, 1: 2164})


In [47]:
from sklearn.linear_model import LogisticRegression

# Example: Training a Logistic Regression model
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_res, y_train_res)

LogisticRegression()

In [48]:
# Predict on test data
y_pred = log_reg_model.predict(X_test)
print(len(y_pred))
print(len(y_test))

1120
1120


In [49]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

    negative       0.53      0.55      0.54       196
     neutral       0.41      0.40      0.40       196
    positive       0.83      0.83      0.83       728

    accuracy                           0.70      1120
   macro avg       0.59      0.59      0.59      1120
weighted avg       0.70      0.70      0.70      1120



In [50]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']  # Solvers
}

In [51]:
# Initialize the Logistic Regression model
log_reg = LogisticRegression(class_weight='balanced')

# Set up the grid search
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='f1_macro', verbose=2)

# Perform grid search on the oversampled training data
grid_search.fit(X_train_res, y_train_res)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END ...........................C=0.01, solver=newton-cg; total time=   0.0s
[CV] END ...........................C=0.01, solver=newton-cg; total time=   0.0s
[CV] END ...........................C=0.01, solver=newton-cg; total time=   0.0s
[CV] END ...........................C=0.01, solver=newton-cg; total time=   0.0s
[CV] END ...........................C=0.01, solver=newton-cg; total time=   0.0s
[CV] END ...............................C=0.01, solver=lbfgs; total time=   0.0s
[CV] END ...............................C=0.01, solver=lbfgs; total time=   0.0s
[CV] END ...............................C=0.01, solver=lbfgs; total time=   0.0s
[CV] END ...............................C=0.01, solver=lbfgs; total time=   0.0s
[CV] END ...............................C=0.01, solver=lbfgs; total time=   0.0s
[CV] END ...........................C=0.01, solver=liblinear; total time=   0.0s
[CV] END ...........................C=0.01, sol

C:\Users\Simon\anaconda3\envs\cuda-pytorch\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Simon\anaconda3\envs\cuda-pytorch\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

[CV] END .................................C=10, solver=lbfgs; total time=   0.0s
[CV] END .................................C=10, solver=lbfgs; total time=   0.0s


C:\Users\Simon\anaconda3\envs\cuda-pytorch\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Simon\anaconda3\envs\cuda-pytorch\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

[CV] END .................................C=10, solver=lbfgs; total time=   0.0s
[CV] END .................................C=10, solver=lbfgs; total time=   0.0s


...

[CV] END .................................C=100, solver=saga; total time=   0.1s
[CV] END .................................C=100, solver=saga; total time=   0.1s


C:\Users\Simon\anaconda3\envs\cuda-pytorch\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Simon\anaconda3\envs\cuda-pytorch\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END .................................C=100, solver=saga; total time=   0.1s


C:\Users\Simon\anaconda3\envs\cuda-pytorch\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=5, estimator=LogisticRegression(class_weight='balanced'),
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='f1_macro', verbose=2)

In [52]:
# Best model
best_model = grid_search.best_estimator_

# Predict and evaluate on the test set
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.46      0.49      0.47       196
           1       0.30      0.30      0.30       196
           2       0.79      0.78      0.78       728

    accuracy                           0.64      1120
   macro avg       0.52      0.52      0.52      1120
weighted avg       0.65      0.64      0.65      1120



In [53]:
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'C': 100, 'solver': 'sag'}


### Word2Vec and LSTM

In [54]:
import gensim.downloader as api

# Load a pre-trained Word2Vec model
word2vec_model = api.load("word2vec-google-news-300")  # or another model of your choice

[--------------------------------------------------] 1.4% 23.4/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=-------------------------------------------------] 3.7% 61.6/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==------------------------------------------------] 5.9% 98.7/1662.8MB downloaded

...

[================================================--] 96.2% 1599.1/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=================================================-] 98.5% 1637.3/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [55]:
rest_train_sentences = [" ".join(item['sentence']) for item in rest_train]
rest_train_polarities = [item['polarity'] for item in rest_train]

rest_test_sentences = [" ".join(item['sentence']) for item in rest_test]
rest_test_polarities = [item['polarity'] for item in rest_test]

In [56]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(rest_train_polarities)
y_test = label_encoder.transform(rest_test_polarities)

In [57]:
import torch

def get_word_vector(word, model):
    try:
        return torch.tensor(model[word])
    except KeyError:
        return torch.zeros(model.vector_size)

def prepare_sequence(sentence, word2vec_model):
    return torch.stack([get_word_vector(word, word2vec_model) for word in sentence.split()])

X_train = [prepare_sequence(sentence, word2vec_model) for sentence in rest_train_sentences]
X_test = [prepare_sequence(sentence, word2vec_model) for sentence in rest_test_sentences]

In [58]:
from torch.nn.utils.rnn import pad_sequence

X_train_padded = pad_sequence(X_train, batch_first=True)
X_test_padded = pad_sequence(X_test, batch_first=True)

In [59]:
# Assuming 'labels' is a list of integer labels
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [65]:
import torch.nn as nn

class SentimentLSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim=3):
        super(SentimentLSTM, self).__init__()
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # Get the output of the last time step
        out = self.fc(lstm_out)
        return out

In [63]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [66]:
# Initialize the model
model = SentimentLSTM(embedding_dim=300, hidden_dim=50)
model.to(device)

SentimentLSTM(
  (lstm): LSTM(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=3, bias=True)
)

In [67]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### BERT

In [15]:
import torch

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [23]:
rest_train_sentences = [" ".join(item['sentence']) for item in rest_train]
rest_train_polarities = [item['polarity'] for item in rest_train]

rest_test_sentences = [" ".join(item['sentence']) for item in rest_test]
rest_test_polarities = [item['polarity'] for item in rest_test]

In [24]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(rest_train_polarities)
y_test = label_encoder.transform(rest_test_polarities)

In [25]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_sentences(sentences, max_length=512):
    input_ids = []
    attention_masks = []

    for sentence in sentences:
        encoded_dict = tokenizer.encode_plus(
                            sentence,                      # Sentence to encode
                            add_special_tokens=True,       # Add '[CLS]' and '[SEP]'
                            max_length=max_length,         # Pad & truncate all sentences
                            padding='max_length',          # Pad all sentences to max length
                            truncation=True,               # Explicitly truncate to max length
                            return_attention_mask=True,    # Construct attention masks
                            return_tensors='pt',           # Return pytorch tensors
                        )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Encode the sentences (X_train and X_test)
train_inputs, train_masks = encode_sentences(rest_train_sentences)
test_inputs, test_masks = encode_sentences(rest_test_sentences)

In [26]:
train_labels = torch.tensor(y_train)
test_labels = torch.tensor(y_test)

In [27]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32  # Adjust this according to your GPU capacity

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our test set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [31]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",  # Use the 12-layer BERT model, with an uncased vocab
    num_labels=3,        # Number of output labels (3 for positive/negative/neutral)
    output_attentions=False,
    output_hidden_states=False,
)

# Tell the model to run on GPU
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)  # Adjust learning rate as needed
epochs = 10
# Training loop
for epoch_i in range(0, epochs):
    # Training
    model.train()
    total_loss = 0

    for step, batch in tqdm(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Clear any previously calculated gradients
        model.zero_grad()

        # Perform a forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        
        # Accumulate the training loss
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        # Update parameters and take a step using the computed gradient
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss}")

2it [01:08, 34.42s/it]

In [ ]:
model.eval()

for batch in test_dataloader:
    # Add batch to GPU
    batch = tuple(t.cuda() for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory
